## Second level GLM analysis

This script performs group level modeling of BOLD response. Script features: 
- loads statistical maps from first level GLM analysis
- discard data from excluded subjects
- performs second level GLM analysis

---
**Last update**: 24.07.2020 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import numpy as np
import sys
import os

from nistats.second_level_model import SecondLevelModel

path_root = os.environ.get('DECIDENET_PATH')
path_code = os.path.join(path_root, 'code')
if path_code not in sys.path:
    sys.path.append(path_code)
from dn_utils.behavioral_models import load_behavioral_data
from dn_utils.glm_utils import load_first_level_stat_maps

In [ ]:
# Directory to save second level output
path_out = os.path.join(path_root, 
                        'data/main_fmri_study/derivatives/nistats/second_level_output')
os.makedirs(path_out, exist_ok=True)

# Load behavioral data
path_beh = os.path.join(path_root, 'data/main_fmri_study/sourcedata/behavioral')
beh, meta = load_behavioral_data(path=path_beh, verbose=False)
n_subjects, n_conditions, n_trials, _ = beh.shape

Before constructing second level GLM, data from excluded subjects is discarded. To exclude subjects, filter vectors are used:
- `ok_all`: True / False vector of length `n_subjects` specifying if given subject should be included in second level analysis
- `ok_index`: vector of indices for included subjects
- `n_subjects_ok`: final group size, number of included subjects (`len(ok_index) is n_subjects_ok`)

In [ ]:
path_nistats = os.path.join(path_root, 'data/main_fmri_study/derivatives/nistats')
path_first_level_output = os.path.join(path_nistats, 'first_level_output')
path_exclusion_csv = os.path.join(path_nistats, 'exclusion/exclusion.csv')

# Load exclusion table
df_exclusion = pd.read_csv(path_exclusion_csv, index_col=0)
ok_all = df_exclusion['ok_all']
ok_index = df_exclusion.index[ok_all]
n_subjects_ok = ok_all.sum()

## Group analysis

Here, group analysis is performed for model-based prediction error regressor, outcome onset regressor and button press regressor. Data from subjects excluded from the analysis is discarded at this point. Second level GLMs are created and fitted, then the resultant statistical map is displayed on the MNI152 non-linear asymmetric template (version 2009c; default template for fmriprep).

In [ ]:
# Define second level model
second_level_model = SecondLevelModel(smoothing_fwhm=None)

# Create second level design matrix
effect_con_rew = np.vstack((np.ones((n_subjects_ok, 1)), 
                            np.zeros((n_subjects_ok, 1))))
effect_con_pun = np.vstack((np.zeros((n_subjects_ok, 1)), 
                            np.ones((n_subjects_ok, 1))))
effect_sub = np.vstack((np.eye(n_subjects_ok), np.eye(n_subjects_ok)))
design_matrix = pd.DataFrame(
    np.hstack((effect_con_rew, effect_con_pun, effect_sub)),
    columns=['rew', 'pun'] + [meta['dim1'][i] for i in ok_index]
)

# Compute statistical map for difference between conditions contrast
condition_rew = np.array(design_matrix.columns == 'rew', dtype='float')
condition_pun = np.array(design_matrix.columns == 'pun', dtype='float')

Conditions are modeled separately as different "sessions", within subject effect of repeated measures is also modeled.

### **Effect 1**: *Combined effect of prediction error coding*
- name: `perr_combined_pos` / `perr_combined_neg` 
- queston: Which brain structures scale their BOLD activity according to magnitude of prediction error regardless of outcome valence?

### **Effect 2**: *Difference in prediction error coding depending on outcome valence*
- name: `perr_rew_minus_pun` / `perr_pun_minus_rew` 
- question: Does prediction error processing differ between reward and punishment condition? 

### **Effect 3**: *Combined effect of expected probability of winning*
Which brain structures respond to expected probability of correct choice regardless of outcome valence?

### **Effect 4**: *Combined effect of difference betweem left and right button press*
Which brain structures invoke left button presses and supress their activity for right button press (or vice versa) regardless of outcome valence?

In [ ]:
# Define effects of interest
effects = [
    {
        'name': 'perr_combined_pos',
        'first_level_contrast': 'out_perr',
        'second_level_contrast': condition_pun + condition_rew
    },
    {
        'name': 'perr_combined_neg',
        'first_level_contrast': 'out_perr',
        'second_level_contrast': (-1) * condition_pun - condition_rew
    },
    {
        'name': 'perr_rew_minus_pun',
        'first_level_contrast': 'out_perr',
        'second_level_contrast': condition_rew - condition_pun
    },
    {
        'name': 'perr_pun_minus_rew',
        'first_level_contrast': 'out_perr',
        'second_level_contrast': condition_pun - condition_rew
    }
]

In [ ]:
for effect in effects:
    
    effect_name = effect['name']
    first_level_contrast = effect['first_level_contrast']
    second_level_contrast = effect['second_level_contrast']

    # Load stat-map images (first level output)
    path_contrast = os.path.join(path_first_level_output, first_level_contrast)
    stat_maps = load_first_level_stat_maps(path_contrast, ['prlrew', 'prlpun'])

    # Filter out excluded subjects
    stat_maps_ok = {con: [stat_maps[con][i] for i in ok_index] for con in stat_maps}

    # Create second level GLM input
    second_level_input = [stat_map 
                          for con in ['prlrew', 'prlpun'] 
                          for stat_map in stat_maps_ok[con]]

    # Fit second level model
    second_level_model.fit(second_level_input, design_matrix=design_matrix)

    # Calculalte second level stat map
    stat_map_2nd = second_level_model.compute_contrast(
        second_level_contrast=second_level_contrast,
        output_type='z_score')
    
    stat_map_2nd_fname = f'statmap-2nd_effect-{effect_name}'
    nib.save(stat_map_2nd, os.path.join(path_out, stat_map_2nd_fname))